### 1. 토큰 분류 (Token Classification)



In [1]:
from datasets import load_dataset

# CoNLL-2003 URL이 변경되었음.
#raw_datasets = load_dataset("conll2003", revision="master")
raw_datasets = load_dataset("conll2003")


In [2]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [3]:
raw_datasets["train"][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [4]:
raw_datasets["train"].features

{'id': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'pos_tags': Sequence(feature=ClassLabel(names=['"', "''", '#', '$', '(', ')', ',', '.', ':', '``', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'NN|SYM', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB'], id=None), length=-1, id=None),
 'chunk_tags': Sequence(feature=ClassLabel(names=['O', 'B-ADJP', 'I-ADJP', 'B-ADVP', 'I-ADVP', 'B-CONJP', 'I-CONJP', 'B-INTJ', 'I-INTJ', 'B-LST', 'I-LST', 'B-NP', 'I-NP', 'B-PP', 'I-PP', 'B-PRT', 'I-PRT', 'B-SBAR', 'I-SBAR', 'B-UCP', 'I-UCP', 'B-VP', 'I-VP'], id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)}

In [5]:
ner_feature = raw_datasets["train"].features["ner_tags"]
ner_feature


Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [6]:
print(raw_datasets["train"].features["tokens"])
print(raw_datasets["train"].features["pos_tags"])
print(raw_datasets["train"].features["chunk_tags"])

Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)
Sequence(feature=ClassLabel(names=['"', "''", '#', '$', '(', ')', ',', '.', ':', '``', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'NN|SYM', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB'], id=None), length=-1, id=None)
Sequence(feature=ClassLabel(names=['O', 'B-ADJP', 'I-ADJP', 'B-ADVP', 'I-ADVP', 'B-CONJP', 'I-CONJP', 'B-INTJ', 'I-INTJ', 'B-LST', 'I-LST', 'B-NP', 'I-NP', 'B-PP', 'I-PP', 'B-PRT', 'I-PRT', 'B-SBAR', 'I-SBAR', 'B-UCP', 'I-UCP', 'B-VP', 'I-VP'], id=None), length=-1, id=None)


In [7]:
label_names = ner_feature.feature.names
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [8]:
print(raw_datasets["train"][0]["tokens"])
print(raw_datasets["train"][0]["ner_tags"])

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']
[3, 0, 7, 0, 0, 0, 7, 0, 0]


In [9]:
words = raw_datasets["train"][0]["tokens"]
labels = raw_datasets["train"][0]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)


EU    rejects German call to boycott British lamb . 
B-ORG O       B-MISC O    O  O       B-MISC  O    O 


In [10]:
print(raw_datasets["train"][4])

{'id': '4', 'tokens': ['Germany', "'s", 'representative', 'to', 'the', 'European', 'Union', "'s", 'veterinary', 'committee', 'Werner', 'Zwingmann', 'said', 'on', 'Wednesday', 'consumers', 'should', 'buy', 'sheepmeat', 'from', 'countries', 'other', 'than', 'Britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.'], 'pos_tags': [22, 27, 21, 35, 12, 22, 22, 27, 16, 21, 22, 22, 38, 15, 22, 24, 20, 37, 21, 15, 24, 16, 15, 22, 15, 12, 16, 21, 38, 17, 7], 'chunk_tags': [11, 11, 12, 13, 11, 12, 12, 11, 12, 12, 12, 12, 21, 13, 11, 12, 21, 22, 11, 13, 11, 1, 13, 11, 17, 11, 12, 12, 21, 1, 0], 'ner_tags': [5, 0, 0, 0, 0, 3, 4, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0]}


In [11]:
words = raw_datasets["train"][4]["tokens"]
labels = raw_datasets["train"][4]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)


Germany 's representative to the European Union 's veterinary committee Werner Zwingmann said on Wednesday consumers should buy sheepmeat from countries other than Britain until the scientific advice was clearer . 
B-LOC   O  O              O  O   B-ORG    I-ORG O  O          O         B-PER  I-PER     O    O  O         O         O      O   O         O    O         O     O    B-LOC   O     O   O          O      O   O       O 


In [12]:
words = raw_datasets["train"][0]["tokens"]
labels = raw_datasets["train"][0]["pos_tags"]
label_names = raw_datasets["train"].features["pos_tags"].feature.names
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)


EU  rejects German call to boycott British lamb . 
NNP VBZ     JJ     NN   TO VB      JJ      NN   . 


In [13]:
words = raw_datasets["train"][0]["tokens"]
labels = raw_datasets["train"][0]["chunk_tags"]
label_names = raw_datasets["train"].features["chunk_tags"].feature.names
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)


EU   rejects German call to   boycott British lamb . 
B-NP B-VP    B-NP   I-NP B-VP I-VP    B-NP    I-NP O 


In [14]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


In [15]:
tokenizer.is_fast


True

In [16]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()


['[CLS]',
 'EU',
 'rejects',
 'German',
 'call',
 'to',
 'boycott',
 'British',
 'la',
 '##mb',
 '.',
 '[SEP]']

In [17]:
raw_datasets["train"][0]["tokens"]

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']

In [18]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"])
print(inputs.tokens())
inputs

['[CLS]', 'EU', '[SEP]']


{'input_ids': [[101, 7270, 102], [101, 22961, 102], [101, 1528, 102], [101, 1840, 102], [101, 1106, 102], [101, 21423, 102], [101, 1418, 102], [101, 2495, 12913, 102], [101, 119, 102]], 'token_type_ids': [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0, 0], [0, 0, 0]], 'attention_mask': [[1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1, 1], [1, 1, 1]]}

In [19]:
raw_datasets["train"][0]["tokens"]

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']

In [20]:
tokenizer(["I am a student.", "You are a student."], is_split_into_words=True)

{'input_ids': [101, 146, 1821, 170, 2377, 119, 1192, 1132, 170, 2377, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [21]:
tokenizer("I am a student.", "You are a student.", is_split_into_words=True)

{'input_ids': [101, 146, 1821, 170, 2377, 119, 1192, 1132, 170, 2377, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [22]:
tokenizer("I am a student.", "You are a student.")

{'input_ids': [101, 146, 1821, 170, 2377, 119, 102, 1192, 1132, 170, 2377, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [23]:
tokenizer(["I am a student.", "You are a student."])

{'input_ids': [[101, 146, 1821, 170, 2377, 119, 102], [101, 1192, 1132, 170, 2377, 119, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]]}

In [24]:
tokenizer(["I am a student."])

{'input_ids': [[101, 146, 1821, 170, 2377, 119, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1]]}

In [25]:
tokenizer("I am a student.")

{'input_ids': [101, 146, 1821, 170, 2377, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [26]:
inputs = tokenizer(["I am a student.", "You are a student."])
print(inputs)
print(inputs.tokens())
print(inputs.word_ids())
print(inputs.tokens(1))
print(inputs.word_ids(1))


{'input_ids': [[101, 146, 1821, 170, 2377, 119, 102], [101, 1192, 1132, 170, 2377, 119, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]]}
['[CLS]', 'I', 'am', 'a', 'student', '.', '[SEP]']
[None, 0, 1, 2, 3, 4, None]
['[CLS]', 'You', 'are', 'a', 'student', '.', '[SEP]']
[None, 0, 1, 2, 3, 4, None]


In [27]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 'EU',
 'rejects',
 'German',
 'call',
 'to',
 'boycott',
 'British',
 'la',
 '##mb',
 '.',
 '[SEP]']

In [28]:
inputs.word_ids()

[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, None]

In [29]:
raw_datasets["train"][0]["tokens"]

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']

In [30]:
raw_datasets["train"][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

약간의 부가 작업으로 토큰과 일치되도록 레이블 목록을 확장할 수 있습니다. 적용할 첫번째 규칙은 특수 토큰의 레이블의 ID가 -100이라는 것입니다. 기본적으로 -100은 우리가 사용할 손실 함수(cross entropy)에서 무시되는 인덱스이기 때문입니다. 추가적으로, 위의 토큰 식별자 리스트에서 두번째 7에 해당하는 토큰의 레이블은 첫번째 7에 해당하는 토큰의 레이블과 동일한 값을 가지게 됩니다. 물론 첫번째 7에 해당하는 토큰은 B-로 시작하고 두번째는 I-로 시작하는 레이블로 지정됩니다:

In [31]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # 새로운 단어의 시작 토큰.
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # 특수 토큰.
            new_labels.append(-100)
        else:
            # 이전 토큰과 동일한 단어에 소속된 토큰.
            label = labels[word_id]
            # 만약 레이블이 B-XXX이면 이를 I-XXX로 변경.
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels


In [32]:
def new_align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id is None:
            new_labels.append(-100)
            #current_word = word_id
        elif current_word != word_id:
            label = labels[word_id]
            new_labels.append(label)
            current_word = word_id
        else:
            label = labels[word_id]
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return(new_labels)

In [33]:
# ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 
# 'B-MISC', 'I-MISC']
inputs = tokenizer(raw_datasets["train"][4]["tokens"], is_split_into_words=True)
word_ids = inputs.word_ids()
labels = raw_datasets["train"][4]["ner_tags"]
print(word_ids)
print(labels)
print(align_labels_with_tokens(labels, word_ids))
print(new_align_labels_with_tokens(labels, word_ids))

[None, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 10, 11, 11, 11, 12, 13, 14, 15, 16, 17, 18, 18, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, None]
[5, 0, 0, 0, 0, 3, 4, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0]
[-100, 5, 0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, -100]
[-100, 5, 0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, -100]


보시다시피, 함수는 시작과 끝에 두 개의 특수 토큰에 대해 -100을 추가하고 **두 개의 토큰으로 분할된 토큰에 대해 0을 추가**했습니다.

✏️ Your turn! 일부 연구자는 단어당 하나의 레이블만 지정하고 해당 단어의 다른 하위 토큰에 -100을 할당하는 것을 선호합니다. 이는 손실(loss)에 크게 기여하는 많은 하위 토큰으로 분할되는 긴 단어를 회피하기 위함입니다. 이 규칙에 따라 레이블을 입력 ID와 정렬할 수 있도록 이전 함수를 변경해 봅시다.

In [34]:
print(raw_datasets["train"][4]["tokens"])
inputs = tokenizer(raw_datasets["train"][4]["tokens"], is_split_into_words=True)
print(inputs.tokens())

['Germany', "'s", 'representative', 'to', 'the', 'European', 'Union', "'s", 'veterinary', 'committee', 'Werner', 'Zwingmann', 'said', 'on', 'Wednesday', 'consumers', 'should', 'buy', 'sheepmeat', 'from', 'countries', 'other', 'than', 'Britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.']
['[CLS]', 'Germany', "'", 's', 'representative', 'to', 'the', 'European', 'Union', "'", 's', 'veterinary', 'committee', 'Werner', 'Z', '##wing', '##mann', 'said', 'on', 'Wednesday', 'consumers', 'should', 'buy', 'sheep', '##me', '##at', 'from', 'countries', 'other', 'than', 'Britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.', '[SEP]']


In [35]:
inputs = tokenizer(raw_datasets["train"][4]["tokens"], is_split_into_words=True)
print(inputs.tokens())
word_ids = inputs.word_ids()
print(word_ids)
labels = raw_datasets["train"][4]["ner_tags"]
label_names = raw_datasets["train"].features["ner_tags"].feature.names
print(labels)
print(label_names)
print(align_labels_with_tokens(labels, word_ids))
print(new_align_labels_with_tokens(labels, word_ids))

['[CLS]', 'Germany', "'", 's', 'representative', 'to', 'the', 'European', 'Union', "'", 's', 'veterinary', 'committee', 'Werner', 'Z', '##wing', '##mann', 'said', 'on', 'Wednesday', 'consumers', 'should', 'buy', 'sheep', '##me', '##at', 'from', 'countries', 'other', 'than', 'Britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.', '[SEP]']
[None, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 10, 11, 11, 11, 12, 13, 14, 15, 16, 17, 18, 18, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, None]
[5, 0, 0, 0, 0, 3, 4, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0]
['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']
[-100, 5, 0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, -100]
[-100, 5, 0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, -100]


In [36]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 'EU',
 'rejects',
 'German',
 'call',
 'to',
 'boycott',
 'British',
 'la',
 '##mb',
 '.',
 '[SEP]']

In [37]:
inputs

{'input_ids': [101, 7270, 22961, 1528, 1840, 1106, 21423, 1418, 2495, 12913, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [38]:
len(inputs.input_ids)

12

In [39]:
raw_datasets["train"][4]["tokens"]

['Germany',
 "'s",
 'representative',
 'to',
 'the',
 'European',
 'Union',
 "'s",
 'veterinary',
 'committee',
 'Werner',
 'Zwingmann',
 'said',
 'on',
 'Wednesday',
 'consumers',
 'should',
 'buy',
 'sheepmeat',
 'from',
 'countries',
 'other',
 'than',
 'Britain',
 'until',
 'the',
 'scientific',
 'advice',
 'was',
 'clearer',
 '.']

In [40]:
inputs = tokenizer(raw_datasets["train"][4]["tokens"], is_split_into_words=True)

In [41]:
print(inputs.tokens())
print(inputs.word_ids())
print(raw_datasets["train"][4]["ner_tags"])

['[CLS]', 'Germany', "'", 's', 'representative', 'to', 'the', 'European', 'Union', "'", 's', 'veterinary', 'committee', 'Werner', 'Z', '##wing', '##mann', 'said', 'on', 'Wednesday', 'consumers', 'should', 'buy', 'sheep', '##me', '##at', 'from', 'countries', 'other', 'than', 'Britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.', '[SEP]']
[None, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 10, 11, 11, 11, 12, 13, 14, 15, 16, 17, 18, 18, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, None]
[5, 0, 0, 0, 0, 3, 4, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0]


전체 데이터셋을 전처리하기 위해, 모든 입력을 토큰화하고 모든 레이블에 align_labels_with_tokens()를 적용해야 합니다. "빠른(fast)" 토크나이저의 장점을 활용하려면 다량의 텍스트를 동시에 토큰화하는 것이 가장 좋습니다. 따라서 예제 리스트를 처리하는 함수를 작성하고 batched=True 옵션을 지정하여 Dataset.map() 메서드를 사용합니다. 이전 예제와 다른 점은 한번에 다중 텍스트가 포함된 배치(batch)를 처리해야 하므로, word_ids() 함수의 매개변수로 현재 배치(batch)에 대한 인덱스를 매개변수로 넘겨야 한다는 것입니다. 따라서 다음과 같이 새로운 함수를 작성합니다:

In [42]:
l = []
l.append([1,2])
print(l)
l.append([2,3,4])
print(l)

[[1, 2]]
[[1, 2], [2, 3, 4]]


In [43]:
import torch
# examples는 단일 텍스트(문장)가 아니라 다중 텍스트임.
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    # print(tokenized_inputs["input_ids"]) # 1000개씩 처리
    all_labels = examples["ner_tags"]
    #print(len(all_labels)) # 1000개 1000개 1000개 ... 1000개 41개 총 14041개
    #[[3, 0, 7, 0, 0, 0, 7, 0, 0], [1, 2], [5, 0], [0, 3, 4, 0, 0, 0, 0, 
    #0, 0, 7, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
    #0],...]
    new_labels = []
    for i, labels in enumerate(all_labels): #1000개씩 처리
        word_ids = tokenized_inputs.word_ids(i)    # 배치(batch) 인덱스 지정
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs


In [44]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)


Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [45]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3453
    })
})

In [46]:
print(len(tokenized_datasets["train"]["input_ids"])) # or, print(len(tokenized_datasets["train"])
print(tokenized_datasets["train"][:2])

14041
{'input_ids': [[101, 7270, 22961, 1528, 1840, 1106, 21423, 1418, 2495, 12913, 119, 102], [101, 1943, 14428, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1]], 'labels': [[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100], [-100, 1, 2, -100]]}


In [47]:
import torch
# examples는 단일 텍스트(문장)가 아니라 다중 텍스트임.
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    # print(tokenized_inputs["input_ids"]) # 1000개씩 처리
    all_labels = examples["ner_tags"]
    #print(len(all_labels)) # 1000개 1000개 1000개 ... 1000개 41개 총 14041개
    #[[3, 0, 7, 0, 0, 0, 7, 0, 0], [1, 2], [5, 0], [0, 3, 4, 0, 0, 0, 0, 
    #0, 0, 7, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
    #0],...]
    new_labels = []
    for i, labels in enumerate(all_labels): #1000개씩 처리
        word_ids = tokenized_inputs.word_ids(i)    # 배치(batch) 인덱스 지정
        new_labels.append(new_align_labels_with_tokens(labels, word_ids)) # the new_align_labels_with_tokens() function is called

    tokenized_inputs["labels"] = new_labels
    #return tokenized_inputs
    return {
        "input_ids" : tokenized_inputs["input_ids"],
        "token_type_ids" : tokenized_inputs["token_type_ids"],
        "attention_mask" : tokenized_inputs["attention_mask"],
        "labels" : new_labels
    }

tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

print(len(tokenized_datasets["train"]["input_ids"])) # or, print(len(tokenized_datasets["train"])
print(tokenized_datasets["train"][:2])

14041
{'input_ids': [[101, 7270, 22961, 1528, 1840, 1106, 21423, 1418, 2495, 12913, 119, 102], [101, 1943, 14428, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1]], 'labels': [[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100], [-100, 1, 2, -100]]}


### Trainer API를 이용하여 모델 미세조정
Trainer를 활용하는 실제 코드는 이전과 동일합니다. 유일한 변경 사항은 데이터가 배치(batch)로 조합되는 방식과 메트릭(metrics) 계산 기능입니다.

#### 데이터 콜레이션 (Data collation)
앞서서 살펴봤던, **DataCollatorWithPadding은 입력(input IDs, attention mask 및 token type IDs)에 대해서만 패딩(padding)을 수행**하기 때문에 여기서는 사용할 수 없습니다. 여기서는 레이블도 입력과 똑같은 방식으로 패딩(padding)이 되어야 동일한 크기를 유지하고 -100을 패딩값으로 사용하여 해당 예측이 손실 계산에서 무시되도록 할 수 있습니다. 또한 datasets을 tensor로 바꾸어 줌. 

이는 모두 DataCollatorForTokenClassification에 의해 수행됩니다. DataCollatorWithPadding과 마찬가지로 입력을 전처리하는데 사용되는 tokenizer를 사용합니다:

In [48]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)


In [49]:
print(tokenized_datasets["train"][0])
print(tokenized_datasets["train"][:3])
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
print(batch["labels"])
print(batch)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': [101, 7270, 22961, 1528, 1840, 1106, 21423, 1418, 2495, 12913, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]}
{'input_ids': [[101, 7270, 22961, 1528, 1840, 1106, 21423, 1418, 2495, 12913, 119, 102], [101, 1943, 14428, 102], [101, 26660, 13329, 12649, 15928, 1820, 118, 4775, 118, 1659, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100], [-100, 1, 2, -100], [-100, 5, 6, 6, 6, 0, 0, 0, 0, 0, -100]]}
tensor([[-100,    3,    0,    7,    0,    0,    0,    7,    0,    0,    0, -100],
        [-100,    1,    2, -100, -100, -100, -100, -100, -100, -100, -100, -100]])
{'input_ids': tensor([[  101,  7270, 22961,  1528,  1840,  1106, 21

In [50]:
def collate_fn(examples):
        return tokenizer.pad(
            examples,
            padding="longest",
            max_length=max_length,
            #pad_to_multiple_of=pad_to_multiple_of,
            return_tensors="pt",
)
    
def collate_fn_1(examples):
        return tokenizer(
            examples,
            padding="longest",
            max_length=max_length,
            #pad_to_multiple_of=pad_to_multiple_of,
            return_tensors="pt",
)    

In [51]:
print(tokenized_datasets["train"][2])
batch = collate_fn(tokenized_datasets["train"][2])
print(batch)

{'input_ids': [101, 26660, 13329, 12649, 15928, 1820, 118, 4775, 118, 1659, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, 5, 6, 6, 6, 0, 0, 0, 0, 0, -100]}
{'input_ids': tensor([  101, 26660, 13329, 12649, 15928,  1820,   118,  4775,   118,  1659,
          102]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), 'labels': tensor([-100,    5,    6,    6,    6,    0,    0,    0,    0,    0, -100])}


In [52]:
print(raw_datasets["train"][2])
batch = collate_fn_1(raw_datasets["train"][2]["tokens"][0])
#batch = collate_fn_1(raw_datasets["train"][:2])
print(batch)

{'id': '2', 'tokens': ['BRUSSELS', '1996-08-22'], 'pos_tags': [22, 11], 'chunk_tags': [11, 12], 'ner_tags': [5, 0]}
{'input_ids': tensor([[  101, 26660, 13329, 12649, 15928,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}


In [3]:
# 일반적인 경우...
#!pip install seqeval

# 아나콘다 가상환경을 사용하는 경우...
# https://stackoverflow.com/questions/59997065/pip-python-normal-site-packages-is-not-writeable/65290638
#!python3 -m pip install seqeval

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [75]:
#from datasets import load_metric
#metric = load_metric("seqeval")
import evaluate
metric = evaluate.load("seqeval")

In [55]:
#!env | grep CONDA

CONDA_SHLVL=1
CONDA_PROMPT_MODIFIER=(transformer) 
CONDA_EXE=/scratch/qualis/miniconda3/bin/conda
_CE_CONDA=
CONDA_PREFIX=/scratch/qualis/miniconda3/envs/transformer
CONDA_PYTHON_EXE=/scratch/qualis/miniconda3/bin/python
CONDA_DEFAULT_ENV=transformer


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [84]:
label_names = raw_datasets["train"].features["ner_tags"].feature.names
label= raw_datasets["train"][0]["ner_tags"]
print(labels)
labels = [label_names[i] for i in labels]
print(labels)

[3, 0, 7, 0, 0, 0, 7, 0, 0]
['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']


In [87]:
predictions = labels.copy()
predictions[2] = "0"
print(predictions)
print(labels)
metric.compute(predictions=[predictions], references=[labels])
#metric.compute(predictions=predictions, references=labels) #error

['B-ORG', 'O', '0', 'O', 'O', 'O', 'B-MISC', 'O', 'O']
['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']


{'MISC': {'precision': 1.0,
  'recall': 0.5,
  'f1': 0.6666666666666666,
  'number': 2},
 'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 0.6666666666666666,
 'overall_f1': 0.8,
 'overall_accuracy': 0.8888888888888888}

In [88]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    print(logits, labels)
    predictions = np.argmax(logits, axis=-1)

    # 무시된 인덱스(특수 토큰들)를 제거하고 레이블로 변환
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }



#### 모델 정의하기
토큰 분류 문제에 대해 작업 중이므로 AutoModelForTokenClassification 클래스를 사용합니다. 이 모델을 정의할 때 기억해야 할 주요 사항은 가지고 있는 레이블 수에 대한 정보를 전달하는 것입니다. 이를 수행하는 가장 쉬운 방법은 num_labels 인수를 사용하여 해당 값을 전달하는 것이지만, 이 섹션의 시작 부분에서처럼 작동하는 멋진 추론 위젯(inference widget)을 원한다면, 올바른 레이블 대응 정보(label correspondences)를 설정하는 것이 좋습니다.

ID에서 레이블로 또는 그 반대로의 매핑을 포함하는 두 개의 딕셔너리, 즉 id2label 및 label2id를 활용하여 이 대응 정보를 설정할 수 있습니다:

In [98]:
print(type(label_names[0]))
print(label_names)
id2label = {i: label for i, label in enumerate(label_names)}
#id2label = {str(i): label for i, label in enumerate(label_names)}
id2label

<class 'str'>
['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']


{0: 'O',
 1: 'B-PER',
 2: 'I-PER',
 3: 'B-ORG',
 4: 'I-ORG',
 5: 'B-LOC',
 6: 'I-LOC',
 7: 'B-MISC',
 8: 'I-MISC'}

In [99]:
label2id = {v:k for k, v in id2label.items()}
label2id

{'O': 0,
 'B-PER': 1,
 'I-PER': 2,
 'B-ORG': 3,
 'I-ORG': 4,
 'B-LOC': 5,
 'I-LOC': 6,
 'B-MISC': 7,
 'I-MISC': 8}

In [96]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    #id2label=id2label,
    #label2id=label2id,
)
print(model)
print(model.config)
print(model.config.num_labels)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [100]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)
print(model)
print(model.config)
print(model.config.num_labels)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [109]:
print(f">>> BERT number of parameters: {model.num_parameters() / 1_000_000}M") # bert-base-cased

>>> BERT number of parameters: 107.726601M


In [101]:
from huggingface_hub import notebook_login

notebook_login()


In [102]:
from transformers import TrainingArguments

args = TrainingArguments(
    "bert-finetuned-ner",
    evaluation_strategy="epoch",
    #evaluation_strategy="steps",
    #save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    #push_to_hub=True,
)


In [103]:
import numpy as np

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    #print(logits.shape, labels.shape) # (3250, 151, 9) (3250, 151) validation datasets size
    predictions = np.argmax(logits, axis=-1)
    #print(predictions.shape) # (3250, 151)

    # 무시된 인덱스(특수 토큰들)를 제거하고 레이블로 변환
    #print(predictions[:2])
    #print(labels[:2])
    #print([[label_names[l] for l in label if l != -100] for label in labels[:5]])
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    #print(true_labels[:2])
    #[['O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 
    #  'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 
    #  'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'O', 'O', 
    #  'O', 'O', 'O']]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    #print(true_predictions[:2])
    #[['O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 
    #  'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'
    #  'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'O', 'O', 
    #  'O', 'O', 'O']]
    
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }



In [104]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()


Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.074400,0.060147,0.915380,0.937563,0.926339,0.983340
2,0.033600,0.070046,0.935291,0.948670,0.941933,0.985607
3,0.023700,0.064511,0.935031,0.951868,0.943374,0.986136


TrainOutput(global_step=5268, training_loss=0.06575343449548418, metrics={'train_runtime': 188.9464, 'train_samples_per_second': 222.936, 'train_steps_per_second': 27.881, 'total_flos': 920580703084350.0, 'train_loss': 0.06575343449548418, 'epoch': 3.0})

학습이 진행되는 동안 모델이 저장될 때마다(여기서는 모든 에포크마다) 백그라운드에서 허브에 업로드됩니다. 이렇게 하면 필요한 경우 다른 서버에서 학습을 재개할 수 있습니다.

학습이 완료되면 push_to_hub() 메서드를 사용하여 모델의 최신 버전을 업로드했는지 확인합니다:

In [63]:
#trainer.push_to_hub(commit_message="Training complete")

또한 Trainer는 모든 평가 결과가 포함된 모델 카드(model card)의 초안을 작성하여 업로드합니다. 이 단계에서 Model Hub의 추론 위젯을 사용하여 모델을 테스트하고 친구와 공유할 수 있습니다. 토큰 분류 작업에서 모델을 성공적으로 미세 조정했습니다. 축하합니다!

In [105]:
%%time
import numpy as np
from torch.optim import AdamW
from torch.utils.data import DataLoader

from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
from datasets import load_dataset, load_metric

from pprint import pprint

# CoNLL-2003 URL이 변경되었음.
#raw_datasets = load_dataset("conll2003", revision="master")
raw_datasets = load_dataset("conll2003")
"""
DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})
"""
model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # 새로운 단어의 시작 토큰.
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # 특수 토큰.
            new_labels.append(-100)
        else:
            # 이전 토큰과 동일한 단어에 소속된 토큰.
            label = labels[word_id]
            # 만약 레이블이 B-XXX이면 이를 I-XXX로 변경.
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

# examples는 단일 텍스트(문장)가 아니라 다중 텍스트임.
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)    # 배치(batch) 인덱스 지정
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)
"""
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3453
    })
})
"""
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

metric = load_metric("seqeval")

train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

eval_dataloader = DataLoader(
    tokenized_datasets["validation"],
    collate_fn=data_collator,
    batch_size=8,
)

label_names = raw_datasets["train"].features["ner_tags"].feature.names
#print(label_names)
id2label = {str(i): label for i, label in enumerate(label_names)}
"""
{'0': 'O', '1': 'B-PER', '2': 'I-PER', '3': 'B-ORG', '4': 'I-ORG', 
'5': 'B-LOC', '6': 'I-LOC', '7': 'B-MISC', '8': 'I-MISC'}
"""
label2id = {v: k for k, v in id2label.items()}

id2label = {i: label for i, label in enumerate(label_names)}
label2id = {k: v for k, v in id2label.items()}

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    #print(logits.shape, labels.shape) # (3250, 151, 9) (3250, 151)
    predictions = np.argmax(logits, axis=-1)

    # 무시된 인덱스(특수 토큰들)를 제거하고 레이블로 변환
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    # print(len(true_predictions), len(true_labels)) (3250, 3250)
    #print(true_predictions)
    #print(labels)
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

args = TrainingArguments(
    "bert-finetuned-ner",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size = 32, # default: 8
    per_device_eval_batch_size = 32, # default: 8
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    #push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

#save
#trainer.save_model()

#eval
metrics = trainer.evaluate(eval_dataset=tokenized_datasets["test"])
pprint(metrics)

#from huggingface_hub import notebook_login
#notebook_login()
#trainer.push_to_hub(commit_message="Training complete")


/scratch/qualis/miniconda3/envs/transformer/lib/python3.10/site-packages/datasets/load.py:1429: FutureWarning: The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

/scratch/qualis/miniconda3/envs/transformer/lib/python3.10/site-packages/datasets/load.py:752: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the ke

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.068491,0.876543,0.920061,0.897775,0.980397
2,0.187600,0.061013,0.912972,0.935712,0.924202,0.983664
3,0.047900,0.055713,0.923813,0.942780,0.933200,0.985047


{'epoch': 3.0,
 'eval_accuracy': 0.9712106648177621,
 'eval_f1': 0.8926808142052836,
 'eval_loss': 0.14742746949195862,
 'eval_precision': 0.8738341529591318,
 'eval_recall': 0.9123583569405099,
 'eval_runtime': 3.4335,
 'eval_samples_per_second': 1005.67,
 'eval_steps_per_second': 31.454}
CPU times: user 1min 27s, sys: 31.8 s, total: 1min 59s
Wall time: 2min 11s


모든 프로세스가 이 지점에 도달할때까지 기다리도록 지시합니다(accelerator.wait_for_everyone()). 이는 저장하기 전에 모든 프로세스에서 동일한 모델이 있는지 확인하기 위한 것입니다. 그런 다음 우리가 정의한 기본 모델인 unwrapped_model을 가져옵니다. accelerator.prepare() 메서드는 모델을 분산 학습(distributed training)에서 작동하도록 변경하므로 모델에 더이상 save_pretrained() 메서드가 없습니다. accelerator.unwrap_model() 메서드는 모델을 다시 save_pretrained()가 존재하는 일반 모델 객체로 돌려놓습니다. 마지막으로 save_pretrained()를 호출하지만 그 메서드에 torch.save() 대신 accelerate.save()를 사용하도록 지시합니다.

In [106]:
%%time
import torch
import numpy as np
from torch.optim import AdamW
from torch.utils.data import DataLoader

from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification, get_scheduler
from datasets import load_dataset, load_metric

from accelerate import Accelerator
from pprint import pprint

from huggingface_hub import Repository, get_full_repo_name

from tqdm.auto import tqdm

# CoNLL-2003 URL이 변경되었음.
#raw_datasets = load_dataset("conll2003", revision="master")
raw_datasets = load_dataset("conll2003")
"""
DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})
"""
model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # 새로운 단어의 시작 토큰.
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # 특수 토큰.
            new_labels.append(-100)
        else:
            # 이전 토큰과 동일한 단어에 소속된 토큰.
            label = labels[word_id]
            # 만약 레이블이 B-XXX이면 이를 I-XXX로 변경.
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

# examples는 단일 텍스트(문장)가 아니라 다중 텍스트임.
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)    # 배치(batch) 인덱스 지정
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)
"""
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3453
    })
})
"""
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

metric = load_metric("seqeval")

train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

eval_dataloader = DataLoader(
    tokenized_datasets["validation"],
    collate_fn=data_collator,
    batch_size=8,
)

label_names = raw_datasets["train"].features["ner_tags"].feature.names
#print(label_names)
id2label = {str(i): label for i, label in enumerate(label_names)}
"""
{'0': 'O', '1': 'B-PER', '2': 'I-PER', '3': 'B-ORG', '4': 'I-ORG', 
'5': 'B-LOC', '6': 'I-LOC', '7': 'B-MISC', '8': 'I-MISC'}
"""
label2id = {v: k for k, v in id2label.items()}

id2label = {i: label for i, label in enumerate(label_names)}
label2id = {k: v for k, v in id2label.items()}

#from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=32,
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"],
    collate_fn=data_collator,
    batch_size=32,
)

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

optimizer = AdamW(model.parameters(), lr=2e-5)

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)


model_name = "bert-finetuned-ner-accelerate"
repo_name = get_full_repo_name(model_name)
print(repo_name)

output_dir = "bert-finetuned-ner-accelerate"
#repo = Repository(output_dir, clone_from=repo_name) # git lfs 설치 에러러

def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # 학습 (Training)
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # 평가 (Evaluation)
    model.eval()
    for batch in eval_dataloader:
        with torch.no_grad():
            outputs = model(**batch)

        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]
        #print(labels.shape)

        # 취합 대상인 예측(predictions)과 레이블(labels)을 패딩하기 위해서 필요함..
        #predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
        predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=0)
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)
        #print(f"***{labels.shape}")

        predictions_gathered = accelerator.gather(predictions)
        labels_gathered = accelerator.gather(labels)

        true_predictions, true_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=true_predictions, references=true_labels)

    results = metric.compute()
    #accelerator.print(f"epoch {epoch}:", results)
    print(
        f"epoch {epoch}:",
        {
            key: results[f"overall_{key}"]
            for key in ["precision", "recall", "f1", "accuracy"]
       },
    )

    # 저장 및 업로드
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        #repo.push_to_hub(
        #    commit_message = f"Training in progress epoch {epoch}", blocking=False,
        #)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

/scratch/qualis/miniconda3/envs/transformer/lib/python3.10/site-packages/datasets/load.py:752: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the ke

qualis2006/bert-finetuned-ner-accelerate


  0%|          | 0/1317 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


epoch 0: {'precision': 0.9173678895994615, 'recall': 0.8743984600577478, 'f1': 0.8953679369250985, 'accuracy': 0.9801612998175075}
[2024-03-22 19:20:07,880] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
epoch 1: {'precision': 0.9382362840794345, 'recall': 0.9158863150977493, 'f1': 0.9269265940643445, 'accuracy': 0.9843704008948019}
epoch 2: {'precision': 0.9419387411645911, 'recall': 0.9198027937551356, 'f1': 0.9307391702003825, 'accuracy': 0.9850473891799612}
CPU times: user 1min 21s, sys: 31.8 s, total: 1min 53s
Wall time: 2min 1s


In [3]:
%%time
import torch
import numpy as np
from torch.optim import AdamW
from torch.utils.data import DataLoader

from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification, get_scheduler
from datasets import load_dataset, load_metric

from accelerate import Accelerator
from pprint import pprint

from huggingface_hub import Repository, get_full_repo_name
from tqdm.auto import tqdm

# CoNLL-2003 URL이 변경되었음.
#raw_datasets = load_dataset("conll2003", revision="master")
raw_datasets = load_dataset("conll2003")
"""
DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})
"""

model_checkpoint = "bert-base-cased"

def get_dataloaders(accelerator: Accelerator, batch_size: int = 8):
    # CoNLL-2003 URL이 변경되었음.
    #raw_datasets = load_dataset("conll2003", revision="master")
    #raw_datasets = load_dataset("conll2003")
    """
    DatasetDict({
        train: Dataset({
            features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
            num_rows: 14041
        })
        validation: Dataset({
            features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
            num_rows: 3250
        })
        test: Dataset({
            features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
            num_rows: 3453
        })
    })
    """
    
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

    def align_labels_with_tokens(labels, word_ids):
        new_labels = []
        current_word = None
        for word_id in word_ids:
            if word_id != current_word:
                # 새로운 단어의 시작 토큰.
                current_word = word_id
                label = -100 if word_id is None else labels[word_id]
                new_labels.append(label)
            elif word_id is None:
                # 특수 토큰.
                new_labels.append(-100)
            else:
                # 이전 토큰과 동일한 단어에 소속된 토큰.
                label = labels[word_id]
                # 만약 레이블이 B-XXX이면 이를 I-XXX로 변경.
                if label % 2 == 1:
                    label += 1
                new_labels.append(label)

        return new_labels

    # examples는 단일 텍스트(문장)가 아니라 다중 텍스트임.
    def tokenize_and_align_labels(examples):
        tokenized_inputs = tokenizer(
            examples["tokens"], truncation=True, is_split_into_words=True
        )
        all_labels = examples["ner_tags"]
        new_labels = []
        for i, labels in enumerate(all_labels):
            word_ids = tokenized_inputs.word_ids(i)    # 배치(batch) 인덱스 지정
            new_labels.append(align_labels_with_tokens(labels, word_ids))

        tokenized_inputs["labels"] = new_labels
        return tokenized_inputs
    
    # Apply the method we just defined to all the examples in all the splits of the dataset
    # starting with the main process first:
    with accelerator.main_process_first():
        tokenized_datasets = raw_datasets.map(
            tokenize_and_align_labels,
            batched=True,
            remove_columns=raw_datasets["train"].column_names,
        )
    """
    DatasetDict({
        train: Dataset({
            features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
            num_rows: 14041
        })
        validation: Dataset({
            features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
            num_rows: 3250
        })
        test: Dataset({
            features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
            num_rows: 3453
        })
    })
    """
    data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

    train_dataloader = DataLoader(
        tokenized_datasets["train"],
        shuffle=True,
        collate_fn=data_collator,
        batch_size=batch_size,
    )
    eval_dataloader = DataLoader(
        tokenized_datasets["validation"],
        collate_fn=data_collator,
        batch_size=batch_size,
    )
    return train_dataloader, eval_dataloader, tokenizer

def training_function():
    accelerator = Accelerator()
    
    train_dataloader, eval_dataloader, tokenizer = get_dataloaders(accelerator, batch_size=32)
    
    label_names = raw_datasets["train"].features["ner_tags"].feature.names
    #print(label_names)
    id2label = {str(i): label for i, label in enumerate(label_names)}
    """
    {'0': 'O', '1': 'B-PER', '2': 'I-PER', '3': 'B-ORG', '4': 'I-ORG', 
    '5': 'B-LOC', '6': 'I-LOC', '7': 'B-MISC', '8': 'I-MISC'}
    """
    label2id = {v: k for k, v in id2label.items()}
    
    id2label = {i: label for i, label in enumerate(label_names)}
    label2id = {k: v for k, v in id2label.items()}
    
    model = AutoModelForTokenClassification.from_pretrained(
        model_checkpoint,
        id2label=id2label,
        label2id=label2id,
    )
    
    optimizer = AdamW(model.parameters(), lr=2e-5)

    model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
        model, optimizer, train_dataloader, eval_dataloader
    )

    num_train_epochs = 3
    num_update_steps_per_epoch = len(train_dataloader)
    num_training_steps = num_train_epochs * num_update_steps_per_epoch

    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps,
    )

    #model_name = "bert-finetuned-ner-accelerate"
    #repo_name = get_full_repo_name(model_name)
    #print(repo_name)
    output_dir = "bert-finetuned-ner-accelerate"
    #repo = Repository(output_dir, clone_from=repo_name) # git lfs 설치 에러 발생

    def postprocess(predictions, labels):
        predictions = predictions.detach().cpu().clone().numpy()
        labels = labels.detach().cpu().clone().numpy()

        # Remove ignored index (special tokens) and convert to labels
        true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
        true_predictions = [
            [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]
        return true_labels, true_predictions

    metric = load_metric("seqeval")
    
    progress_bar = tqdm(range(num_training_steps))

    for epoch in range(num_train_epochs):
    # 학습 (Training)
        model.train()
        for batch in train_dataloader:
            outputs = model(**batch)
            loss = outputs.loss
            accelerator.backward(loss)

            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)

        # 평가 (Evaluation)
        model.eval()
        for batch in eval_dataloader:
            with torch.no_grad():
                outputs = model(**batch)

            predictions = outputs.logits.argmax(dim=-1)
            labels = batch["labels"]
            #print(labels.shape) # batch size 32, number of processes 4
            #torch.Size([32, 52]) torch.Size([32, 49]) torch.Size([32, 18]) torch.Size([32, 51])

            # 취합 대상인 예측(predictions)과 레이블(labels)을 패딩하기 위해서 필요함..
            predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
            #predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=0)
            labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)
            #print(f"*{labels.shape}")
            #*torch.Size([32, 52]) *torch.Size([32, 52]) *torch.Size([32, 52]) *torch.Size([32, 52])
            

            predictions_gathered = accelerator.gather(predictions)
            labels_gathered = accelerator.gather(labels)
            #print(f"**{labels_gathered.shape}")
            #**torch.Size([128, 52]) **torch.Size([128, 52]) **torch.Size([128, 52]) **torch.Size([128, 52])

            true_predictions, true_labels = postprocess(predictions_gathered, labels_gathered)
            #true_predictions, true_labels = postprocess(predictions, labels)
            metric.add_batch(predictions=true_predictions, references=true_labels)

        results = metric.compute()
        # Use accelerator.print to print only on the main process.
        #accelerator.print(f"epoch {epoch}:", results)
        #print(
        accelerator.print(
            f"epoch {epoch}:",
            {
                key: results[f"overall_{key}"]
                for key in ["precision", "recall", "f1", "accuracy"]
           },
        )

        # 저장 및 업로드
        accelerator.wait_for_everyone()
        unwrapped_model = accelerator.unwrap_model(model)
        unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
        if accelerator.is_main_process:
            tokenizer.save_pretrained(output_dir)
            #repo.push_to_hub(
            #    commit_message = f"Training in progress epoch {epoch}", blocking=False,
            #)

#model_name = "bert-finetuned-ner-accelerate"
#repo_name = get_full_repo_name(model_name)
#print(repo_name)
#repo = Repository(output_dir, clone_from=repo_name) # git lfs 설치 에러러

#notebook_launcher(function, args, num_processes, mixed_precision, use_port, master_addr, node_rank, num_nodes)
from accelerate import notebook_launcher

notebook_launcher(training_function, num_processes=4)
        
#def main():
#    parser = argparse.ArgumentParser(description="Simple example of training script.")
#    parser.add_argument(
#        "--mixed_precision",
#        type=str,
#        default=None,
#        choices=["no", "fp16", "bf16", "fp8"],
#        help="Whether to use mixed precision. Choose"
#        "between fp16 and bf16 (bfloat16). Bf16 requires PyTorch >= 1.10."
#        "and an Nvidia Ampere GPU.",
#    )
#    parser.add_argument("--cpu", action="store_true", help="If passed, will train on the CPU.")
#    args = parser.parse_args()
#    config = {"lr": 2e-5, "num_epochs": 3, "seed": 42, "batch_size": 16}
#    training_function(config, args)
#
#
#if __name__ == "__main__":
#    main()

Launching training on 4 GPUs.


Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream ta

  0%|          | 0/330 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


  0%|          | 0/330 [00:00<?, ?it/s]

  0%|          | 0/330 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


  0%|          | 0/330 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


epoch 0: {'precision': 0.8080327868852459, 'recall': 0.7341376228775692, 'f1': 0.7693148119244575, 'accuracy': 0.9576704299367655}
epoch 1: {'precision': 0.899344262295082, 'recall': 0.8403799019607843, 'f1': 0.8688628444726005, 'accuracy': 0.9757745502844108}
epoch 2: {'precision': 0.908360655737705, 'recall': 0.8648353363508663, 'f1': 0.8860638042696092, 'accuracy': 0.9785464730170647}
CPU times: user 458 ms, sys: 213 ms, total: 672 ms
Wall time: 57.1 s


In [107]:
from transformers import pipeline

# 아래 내용을 본인의 checkpoint로 변경하시오.
model_checkpoint = "spasis/bert-finetuned-ner"
#token_classifier = pipeline(
#    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
#)

token_classifier = pipeline(
    "token-classification", model=model_checkpoint
)

token_classifier("My name is Sylvain and I work at Hugging Face in Brooklyn.")


[{'entity': 'B-PER',
  'score': 0.9961349,
  'index': 4,
  'word': 'S',
  'start': 11,
  'end': 12},
 {'entity': 'I-PER',
  'score': 0.996436,
  'index': 5,
  'word': '##yl',
  'start': 12,
  'end': 14},
 {'entity': 'I-PER',
  'score': 0.9951277,
  'index': 6,
  'word': '##va',
  'start': 14,
  'end': 16},
 {'entity': 'I-PER',
  'score': 0.99728584,
  'index': 7,
  'word': '##in',
  'start': 16,
  'end': 18},
 {'entity': 'B-ORG',
  'score': 0.9441438,
  'index': 12,
  'word': 'Hu',
  'start': 33,
  'end': 35},
 {'entity': 'I-ORG',
  'score': 0.98225856,
  'index': 13,
  'word': '##gging',
  'start': 35,
  'end': 40},
 {'entity': 'I-ORG',
  'score': 0.9827559,
  'index': 14,
  'word': 'Face',
  'start': 41,
  'end': 45},
 {'entity': 'B-LOC',
  'score': 0.9920243,
  'index': 16,
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]